In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import nnacc
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_dir = '/Users/pierfied/Downloads/demo_data/'

mask = np.loadtxt(data_dir + 'mask_gamma.txt')[:,1].astype(bool)

X_train = np.loadtxt(data_dir + 'X_train.txt')
X_val = np.loadtxt(data_dir + 'X_val.txt')

y_train = np.load(data_dir + 'y_train.npy')[:,mask,1]
y_val = np.load(data_dir + 'y_val.npy')[:,mask,1]

inv_cov = np.load(data_dir + 'invcov_gamma.npy')[:,mask][mask,:]

In [3]:
cov = np.linalg.inv(inv_cov)
sigma = np.sqrt(np.diag(cov))

cov /= sigma[None,:] * sigma[:,None]
y_train /= sigma[None,:]
y_val /= sigma[None,:]

inv_cov = np.linalg.inv(cov)

In [4]:
plow, med, phigh = np.percentile(y_train, [5, 50, 95], axis=0)

bad_inds_train = np.sum(np.abs(y_train - med[None,:]) > 10 * (phigh - plow), axis=-1) > 0
bad_inds_val = np.sum(np.abs(y_val - med[None,:]) > 10 * (phigh - plow), axis=-1) > 0

X_train = X_train[~bad_inds_train,:]
X_val = X_val[~bad_inds_val,:]

y_train = y_train[~bad_inds_train,:]
y_val = y_val[~bad_inds_val,:]

In [5]:
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

X_train = (X_train - X_mean[None,:]) / X_std[None,:]
X_val = (X_val - X_mean[None,:]) / X_std[None,:]

In [6]:
class ArrayDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, i):
        return self.X[i,:], self.y[i,:]

In [7]:
batch_size = 128

train_dataset = ArrayDataset(X_train, y_train)
val_dataset = ArrayDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [8]:
y_mean = torch.tensor(y_train.mean(axis=0))
y_std = torch.tensor(y_train.std(axis=0))

def transform(y):
    return y * y_std[None,:] + y_mean[None,:]

In [9]:
inv_cov_tensor = torch.tensor(inv_cov)

def loss_fn(y_target, y_pred):
    y_pred = transform(y_pred)
    
    delta = y_pred - y_target
    chisq = torch.sum((delta @ inv_cov_tensor) * delta, dim=-1)
    
    loss = torch.mean(chisq) / y_target.shape[-1]
    
    return loss

In [10]:
model = nnacc.Predictor(X_train.shape[-1], y_train.shape[-1])

In [11]:
model.train(train_loader, 10, loss_fn)

  0%|          | 0/10 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/pierfied/opt/anaconda3/envs/karmma/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-360f11068c28>", line 1, in <module>
    model.train(train_loader, 10, loss_fn)
  File "/Users/pierfied/PycharmProjects/nnacc/nnacc/predictor.py", line 48, in train
    self.optim.step()
  File "/Users/pierfied/opt/anaconda3/envs/karmma/lib/python3.9/site-packages/torch/optim/optimizer.py", line 89, in wrapper
    return func(*args, **kwargs)
  File "/Users/pierfied/opt/anaconda3/envs/karmma/lib/python3.9/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/Users/pierfied/opt/anaconda3/envs/karmma/lib/python3.9/site-packages/torch/optim/adam.py", line 108, in step
    F.adam(params_with_grad,
  File "/Users/pierfied/opt/anaconda3/envs/karmma/lib/python3.9/site-packages/torch/opti

TypeError: object of type 'NoneType' has no len()